In [321]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [322]:
PROVINCE_NUMBER = 15
# Read data from csv file 
import pandas as pd

coverages = pd.read_csv("coverages.csv")
population = pd.read_csv("population_cost.csv")
data = coverages[["ID", "Coverage"]].set_index("ID").join(population.set_index("id"), on="ID")
data = data.reset_index(False)
data = data[["ID", "Coverage", "provice", "population", "cost"]]
coverages[:3], population[:3], data[:3]

(   ID      Tỉnh                                           Coverage
 0   1    Hà Nội  1, 8, 17, 19, 24, 26, 25, 30, 35, 33, 34, 27, ...
 1   2  Hà Giang                                        2, 6, 10, 8
 2   4  Cao Bằng                                           4, 6, 20,
    id   provice  population  cost
 0   1    Hà Nội     8246540     1
 1   2  Hà Giang      870240     1
 2   4  Cao Bằng      533085     1,
    ID                                           Coverage   provice  \
 0   1  1, 8, 17, 19, 24, 26, 25, 30, 35, 33, 34, 27, ...    Hà Nội   
 1   2                                        2, 6, 10, 8  Hà Giang   
 2   4                                           4, 6, 20  Cao Bằng   
 
    population  cost  
 0     8246540     1  
 1      870240     1  
 2      533085     1  )

In [323]:
data_np = data.to_numpy()
data_np[:3]

array([[1, '1, 8, 17, 19, 24, 26, 25, 30, 35, 33, 34, 27, 31, 36, 37',
        'Hà Nội', 8246540, 1],
       [2, '2, 6, 10, 8', 'Hà Giang', 870240, 1],
       [4, '4, 6, 20', 'Cao Bằng', 533085, 1]], dtype=object)

In [324]:
city_ = data_np[:,0][:PROVINCE_NUMBER]
coverage_ = data_np[:, 1][:PROVINCE_NUMBER]
cost_ = data_np[:, 4][:PROVINCE_NUMBER]
city_[:5], coverage_[:5], cost_[:15]

(array([1, 2, 4, 6, 10], dtype=object),
 array(['1, 8, 17, 19, 24, 26, 25, 30, 35, 33, 34, 27, 31, 36, 37',
        '2, 6, 10, 8', '4, 6, 20',
        '2, 4, 6, 8, 15, 19, 20, 24, 26, 25, 27', '2, 10, 12'],
       dtype=object),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=object))

In [325]:
input_data = {}
for i in range(len(cost_)):
    coverage_arr = np.array(coverage_[i].split(", "),dtype= int)
    ele = [{cov for cov in coverage_arr}, cost_[i]]
    input_data[i] = ele

input_data

{0: [{1, 8, 17, 19, 24, 25, 26, 27, 30, 31, 33, 34, 35, 36, 37}, 1],
 1: [{2, 6, 8, 10}, 1],
 2: [{4, 6, 20}, 1],
 3: [{2, 4, 6, 8, 15, 19, 20, 24, 25, 26, 27}, 1],
 4: [{2, 10, 12}, 1],
 5: [{11, 14}, 1],
 6: [{1, 2, 6, 8, 15, 17, 19, 24, 25, 26, 27}, 1],
 7: [{11, 14, 15}, 1],
 8: [{10, 12}, 1],
 9: [{6, 8, 14, 15, 17, 19, 25, 26}, 1],
 10: [{1, 8, 15, 17, 19, 24, 25, 26, 27, 30, 33, 34, 35, 36, 37}, 1],
 11: [{1, 6, 8, 15, 17, 19, 20, 24, 25, 26, 27, 30, 31, 33, 35}, 1],
 12: [{4, 6, 19, 20, 22, 24, 27, 30, 31}, 1],
 13: [{1, 6, 8, 17, 19, 20, 22, 24, 25, 26, 27, 30, 31, 33, 34, 35, 36, 37},
  1],
 14: [{1, 6, 8, 15, 17, 19, 24, 25, 26, 27, 30, 33, 35, 36}, 1]}

In [348]:
budget = 30

input_population = population[['id', 'population']].values
input_population = {id:po for id, po in input_population}

TypeError: unhashable type: 'list'

In [346]:
city1, population = gp.multidict(input_population)

city, coverage, cost = gp.multidict(input_data)

# city, coverage, cost = gp.multidict({
#     0: [{0,1}, 4.2],
#     1: [{0,1,5}, 6.1],
#     2: [{2,3}, 5.2],
#     3: [{2,3,4},5.5],
#     4: [{3,4,5}, 4.8],
#     5: [{1,4,5}, 9.2]
# })
population

{1: 8246540,
 2: 870240,
 4: 533085,
 6: 316463,
 8: 792900,
 10: 746355,
 11: 613480,
 12: 469811,
 14: 1270556,
 15: 831586,
 17: 861216,
 19: 1307871,
 20: 788706,
 22: 1337605,
 24: 1841624,
 25: 1481884,
 26: 1171232,
 27: 1419126,
 30: 1916774,
 31: 2053493,
 33: 1269090,
 34: 1870240,
 35: 861832,
 36: 1780333,
 37: 993920,
 38: 3664944,
 40: 3365198,
 42: 1296622,
 44: 901984,
 45: 637265,
 46: 1133713,
 48: 1169480,
 49: 1505042,
 51: 1233396,
 52: 1487903,
 54: 874295,
 56: 1240436,
 58: 593644,
 60: 1239256,
 62: 555645,
 64: 1541829,
 66: 1886937,
 67: 637907,
 68: 1309792,
 70: 1011076,
 72: 1178329,
 74: 2580550,
 75: 3177401,
 77: 1167937,
 79: 9227598,
 80: 1713658,
 82: 1772544,
 83: 1292397,
 84: 1009940,
 86: 1022971,
 87: 1600014,
 89: 1904532,
 91: 1728869,
 92: 1240731,
 93: 729780,
 94: 1195741,
 95: 913481,
 96: 1193894}

In [328]:
m = gp.Model("set_covering1")

build = m.addVars(len(city), vtype = GRB.BINARY, name = "build")
m.addConstrs((gp.quicksum(build[r] for r in city if r in coverage[t]) >= 1 for t in city), name = "Build2cover")
m.setObjective(gp.quicksum(build[t] for t in city), GRB.MINIMIZE)
m.update()
m.write("set_covering1.lp")
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 15 columns and 41 nonzeros
Model fingerprint: 0x20cb72de
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 15 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%


In [329]:
for ct in build.keys():
    if (abs(build[ct].x) > 1e-6):
        print(f"\n Setup a Kho Lien Vung in city {ct}")


 Setup a Kho Lien Vung in city 6

 Setup a Kho Lien Vung in city 8

 Setup a Kho Lien Vung in city 10

 Setup a Kho Lien Vung in city 14


In [335]:
total_population = 0
covered_population = np.zeros(len(city))

for ct in city1:
    total_population += population[ct]
    
for ct in build.keys():
    if abs(build[ct].x) > 1e-6:
        covered_population[ct] = sum(population[k] for k in city if k in coverage[ct])

for ct in range(len(covered_population)):
    if abs(covered_population[ct]) > 1e-6:
        covered_percentage = covered_population[ct]/total_population * 100
        print("Fire station {0} covers {1} % of the population".format(ct, covered_percentage))

print('Covered population percentage %', covered_population * 100)

Fire station 6 covers 10.479463812297125 % of the population
Fire station 8 covers 1.2462927211898118 % of the population
Fire station 10 covers 9.263363945079893 % of the population
Fire station 14 covers 9.58766632931518 % of the population
Covered population percentage % [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0226143e+09 0.0000000e+00 1.2161660e+08 0.0000000e+00
 9.0394400e+08 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.3559030e+08]


In [349]:
total_cost = 0
for ct in range(len(city)):
    if abs(build[ct].x)> 1e-6:
        total_cost += cost[ct]* build[ct].x
print(total_cost, budget)
budget_consumption = round(100* total_cost/budget, 2)

print(f'\n The percentage of budget : {budget_consumption}%')

4.0 30

 The percentage of budget : 13.33%
